# Preparation

In [15]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh# This Python 3 environment comes with many helpful analytics libraries installed

/bin/bash: sh#: command not found
curl: (23) Failure writing output to destination, passed 1378 returned 1349


In [16]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [17]:
def pull_model(local_llm):
    !ollama pull local_llm

In [18]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [19]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [20]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm

# Dati

https://www.kaggle.com/datasets/waalbannyantudre/hate-speech-detection-curated-dataset

https://www.sciencedirect.com/science/article/pii/S2352340922010356

In [22]:
# todo #all'incirca 7.000 testi di test

df = pd.read_csv("/kaggle/input/hate-speech-detection-curated-dataset/HateSpeechDatasetBalanced.csv")

print(len(df))

#label_0 = df[df["Label"]==0] #no hate speech
#label_1 = df[df["Label"]==1] #hate speech
#print(f"Label con valore 0: {len(label_0)}")
#print(f"Label con valore 1: {len(label_1)}")

# Campiona l'1% del DataFrame
sampled_df = df.sample(frac=0.01, random_state=42)

x_test = sampled_df["Content"]
y_test = sampled_df["Label"]

print(len(x_test))
print(len(y_test))

label_0 = sampled_df[sampled_df["Label"]==0] #no hate speech
label_1 = sampled_df[sampled_df["Label"]==1] #hate speech
print(f"Label con valore 0: {len(label_0)}")
print(f"Label con valore 1: {len(label_1)}")

726119
7261
7261
Label con valore 0: 3584
Label con valore 1: 3677


# Hate speech

In [23]:
model = "llama3.1"

start_ollama()
pull_model(model)
start_model(model)

Exception in thread Thread-8 (<lambda>):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_34/2396888887.py", line 2, in <lambda>
  File "/opt/conda/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/opt/conda/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/opt/conda/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
FileNotFoundError: [Errno 2] No such file or directory: 'ollama'


/bin/bash: ollama: command not found


Exception in thread Thread-9 (<lambda>):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_34/1267679444.py", line 2, in <lambda>
  File "/opt/conda/lib/python3.10/subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/opt/conda/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/opt/conda/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_num, err_msg, err_filename)
FileNotFoundError: [Errno 2] No such file or directory: 'ollama'


In [ ]:
prompt = PromptTemplate(
        template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an helpful assistant who has to detect the presence of hate speech.
            Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
            You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. NO PREAMBLE, NO EXPLANATIONS.
            <|eot_id|><|start_header_id|>user<|end_header_id|> 
            Do you think this document contain hate speech? document: {document}.
            <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
        input_variables=["document"],
    )

In [ ]:
def hate_speech_detection(prompt, model):
    llm = ChatOllama(model=model, temperature=0)
    hate_speech_detection = prompt | llm | StrOutputParser()
    return hate_speech_detection

In [ ]:
# Label 0: no-hate speech (no answer)
# Label 1: hate speech (yes answer)

def predict(prompt,model,x_test):
    y_pred = []
    for x in tqdm(x_test):
        answer = hate_speech_detection(prompt,model).invoke({"document":x})
        if answer.lower() == "yes": y_pred.append(1)
        elif answer.lower() == "no": y_pred.append(0)
    return y_pred

y_pred = predict(prompt,model,x_test)

In [ ]:
# Valutazione del modello
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)
    
write_file("/kaggle/working/test_hate_speech", class_report)